In [7]:
import os
import torch
from data_handling import Labels


In [13]:
lab = Labels()

In [14]:
target_label = "disgust"
text_group, index_group = lab.get_label_groupings(target_label)

In [15]:
from collections import defaultdict

target = defaultdict(list)
complement = defaultdict(list)


path = "steering_vectors/raw_means"
for tensor in os.listdir(path):
    layer = int(tensor.split("_")[0][-1])
    label = int(tensor.split("_")[1].split('label')[1].split('.')[0])
    if label in index_group:
        target[layer].append(torch.load(path + "/" + tensor))
        #target.append(torch.load(path + "/" + tensor))
    else:
        complement[layer].append(torch.load(path + "/" + tensor))
        #complement.append(torch.load(path + "/" + tensor))
#combined = torch.stack(target).mean(dim=0)-torch.stack(complement).mean(dim=0)

C:\Users\Bruger\AppData\Local\Temp\ipykernel_2996\2951083830.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  complement[layer].append(torch.load(path + "/" + tensor))
C

In [16]:
combined = dict()
for layer_index in range(len(complement.keys())):
    combined[layer_index] = torch.stack(target[layer_index]).mean(dim=0)-torch.stack(complement[layer_index]).mean(dim=0)

In [17]:
from pathlib import Path
Path(f"steering_vectors/final_vectors/{target_label}").mkdir(parents=True, exist_ok=True)

for layer_index, tensor in combined.items():
    torch.save(tensor,f"steering_vectors/final_vectors/{target_label}/layer{layer_index}_combined.pt")
    
for layer_index, tensor in target.items():
    torch.save(tensor,f"steering_vectors/final_vectors/{target_label}/layer{layer_index}_target.pt")

for layer_index, tensor in complement.items():
    torch.save(tensor,f"steering_vectors/final_vectors/{target_label}/layer{layer_index}_complement.pt")